In [1]:
%matplotlib notebook
import dill
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

In [2]:

filename = '/Users/bpb/Downloads/Archive (1)/positive_mode_isotope_data.pkl'
with open(filename,'rb') as fid:
    data = dill.load(fid)

In [3]:
for i,d in enumerate(data):
    data[i]['rt_peak'] = pd.DataFrame(data[i]['rt_peak'])
    data[i]['peak_height'] = pd.DataFrame(data[i]['peak_height'])

In [4]:
[c.replace('20171221_TS_DB_fullcyc_QE144_50454-738379_','') for c in data[0]['rt_peak'].columns.tolist()]

[u'100_13C_Oxa_D3_1_POS_146.h5',
 u'101_13C_Oxa_D3_2_POS_210.h5',
 u'102_13C_Oxa_D3_3_POS_181.h5',
 u'103_13C_Oxa_D6_1_POS_284.h5',
 u'104_13C_Oxa_D6_2_POS_34.h5',
 u'105_13C_Oxa_D6_3_POS_267.h5',
 u'106_13C_Oxa_D14_1_POS_100.h5',
 u'107_13C_Oxa_D14_2_POS_126.h5',
 u'108_13C_Oxa_D14_3_POS_161.h5',
 u'109_ext_blank_1_POS_B_386.h5',
 u'10_12C_con_D6_1_POS_08.h5',
 u'110_ext_blank_2_POS_204.h5',
 u'11_12C_con_D6_2_POS_264.h5',
 u'12_12C_con_D6_3_POS_215.h5',
 u'13_12C_con_D14_1_POS_37.h5',
 u'14_12C_con_D14_2_POS_363.h5',
 u'15_12C_con_D14_3_POS_63.h5',
 u'16_12C_con_D30_1_POS_276.h5',
 u'17_12C_con_D30_2_POS_20.h5',
 u'18_12C_con_D30_3_POS_B_369.h5',
 u'19_12C_con_D48_1_POS_95.h5',
 u'1_H2O_con_D48_1_POS_66.h5',
 u'20_12C_con_D48_2_POS_218.h5',
 u'22_13C_Cel_D3_1_POS_290.h5',
 u'23_13C_Cel_D6_1_POS_270.h5',
 u'24_13C_Cel_D6_2_POS_273.h5',
 u'25_13C_Cel_D6_3_POS_89.h5',
 u'26_13C_Cel_D14_1_POS_230.h5',
 u'27_13C_Cel_D14_2_POS_72.h5',
 u'28_13C_Cel_D14_3_POS_31.h5',
 u'29_13C_Cel_D30_1_POS

In [5]:
[(i,d['compound']) for i,d in enumerate(data)]

[(0, [u'glycocyamine', 0]),
 (1, [u'D-Glucosamine', 1]),
 (2, [u'L-Allothreonine', 2]),
 (3, [u'creatinine', 3]),
 (4, [u'N-Acetylneuraminic_Acid', 4]),
 (5, [u'2-Aminoisobutyric_acid', 5]),
 (6, [u'1-Aminocyclopropanecarboxylic_acid', 6]),
 (7, [u'dl-3-Aminoisobutyric_acid', 7]),
 (8, [u'5-Aminovaleric_acid', 8]),
 (9, [u'L-ornithine', 9]),
 (10, [u'xanthosine', 10]),
 (11, [u'L-2-Aminoadipic_acid', 11]),
 (12, [u'2deoxycytidine', 12]),
 (13, [u'L-arabitol', 13]),
 (14, [u'14289-34-0', 14]),
 (15, [u'3-carboxy-1-methylpyridinium', 15]),
 (16, [u'L-glutamine', 16]),
 (17, [u'L-threonine', 17]),
 (18, [u'Imidazole-4-acetic_acid', 18]),
 (19, [u'2deoxyadenosine', 19]),
 (20, [u'creatine', 20]),
 (21, [u'ectoine', 21]),
 (22, [u'L-tyrosine', 22]),
 (23, [u'cytidine', 23]),
 (24, [u'L-citrulline', 24]),
 (25, [u'nicotinamide', 25]),
 (26, [u'2deoxyguanosine', 26]),
 (27, [u'adenosine', 27]),
 (28, [u'4-aminobutyric_acid', 28]),
 (29, [u'L-methionine', 29]),
 (30, [u'L-lysine', 30]),
 (31, 

In [6]:
cpd_id = 30

In [7]:
len(data)

61

In [8]:
data[cpd_id].keys()

['rt_peak', 'peak_height', 'compound']

In [9]:
data[cpd_id]['compound']

[u'L-lysine', 30]

In [10]:
data[cpd_id]['rt_peak'].columns[:3]

Index([u'20171221_TS_DB_fullcyc_QE144_50454-738379_100_13C_Oxa_D3_1_POS_146.h5',
       u'20171221_TS_DB_fullcyc_QE144_50454-738379_101_13C_Oxa_D3_2_POS_210.h5',
       u'20171221_TS_DB_fullcyc_QE144_50454-738379_102_13C_Oxa_D3_3_POS_181.h5'],
      dtype='object')

In [11]:
data[cpd_id]['peak_height'].columns[:3]

Index([u'20171221_TS_DB_fullcyc_QE144_50454-738379_100_13C_Oxa_D3_1_POS_146.h5',
       u'20171221_TS_DB_fullcyc_QE144_50454-738379_101_13C_Oxa_D3_2_POS_210.h5',
       u'20171221_TS_DB_fullcyc_QE144_50454-738379_102_13C_Oxa_D3_3_POS_181.h5'],
      dtype='object')

In [12]:
treatments = list(set(['_'.join(c.split('_')[7:9]) for c in data[cpd_id]['peak_height'].columns if 'D' in c]))
treatments

[u'13C_Ami',
 u'13C_Glu',
 u'13C_Lac',
 u'60_POS',
 u'H2O_con',
 u'13C_Xyl',
 u'96_POS',
 u'13C_Pal',
 u'12C_con',
 u'13C_Gly',
 u'13C_Cel',
 u'13C_Oxa',
 u'13C_Van',
 u'ext_blank']

In [13]:
time_points = []
for c in data[cpd_id]['peak_height'].columns:
    try:
        found = re.search('_D\d+_', c).group()
        time_points.append(found.strip('_'))
    except AttributeError:
        pass
time_points = list(set(time_points))
time_points

[u'D14', u'D30', u'D48', u'D6', u'D3', u'D1']

In [14]:
groups = list(set(['_'.join(c.split('_')[7:10]) for c in data[cpd_id]['peak_height'].columns if 'D48' in c]))
groups

[u'13C_Pal_D48',
 u'13C_Van_D48',
 u'13C_Cel_D48',
 u'H2O_con_D48',
 u'12C_con_D48']

In [15]:
peak_height = data[cpd_id]['peak_height']
control = peak_height[[c for c in peak_height.columns if groups[0] in c]].mean(axis=1)
control

147.1128    1.017697e+06
148.1162    7.030222e+04
149.1195    1.283069e+04
150.1229    9.736336e+02
151.1262    0.000000e+00
152.1296    1.615290e+04
153.1329    0.000000e+00
dtype: float64

In [16]:
labeled = peak_height[[c for c in peak_height.columns if groups[2] in c]].mean(axis=1)
labeled

147.1128    738767.677083
148.1162     52596.792969
149.1195      9509.240885
150.1229      1036.475342
151.1262      1257.195516
152.1296     16633.710938
153.1329       743.332275
dtype: float64

In [ ]:
labeled - control

In [17]:
diff_data = {}
for g in groups:
    sub_data = peak_height[[c for c in peak_height.columns if g in c]].mean(axis=1)
    diff_data[g] = sub_data
df = pd.DataFrame(diff_data)
df

,12C_con_D48,13C_Cel_D48,13C_Pal_D48,13C_Van_D48,H2O_con_D48
147.1128,870130.437500,738767.677083,1.017697e+06,807535.562500,482432.500000
148.1162,56566.234375,52596.792969,7.030222e+04,51074.240885,34027.096354
149.1195,5923.273315,9509.240885,1.283069e+04,6811.392171,5410.415771
150.1229,1014.861206,1036.475342,9.736336e+02,0.000000,0.000000
151.1262,0.000000,1257.195516,0.000000e+00,1351.055013,2449.941243
152.1296,16176.509766,16633.710938,1.615290e+04,15990.358724,22510.484375
153.1329,1269.669434,743.332275,0.000000e+00,864.773844,1387.787109


In [19]:
plt.ioff()
for time_point in time_points:
    folder_name = '/Users/bpb/Downloads/20190131_isotopefigs'
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    for cpd_id in range(len(data)):
        peak_height = data[cpd_id]['peak_height']
        diff_data = {}
        fig = plt.figure()
        ax = fig.gca()
        for treatment in treatments:
            g = '%s_%s'%(treatment,time_point)
            sub_data = peak_height[[c for c in peak_height.columns if g in c]].mean(axis=1)
            diff_data[g] = sub_data
        df = pd.DataFrame(diff_data)
        df.index = [d.replace('p','.') for d in df.index]
        df[:-1].plot(kind='bar',ax=ax) #drop the unnecessary last ion
        plt.title(data[cpd_id]['compound'],fontsize=18)
        ax.set_yscale('log')
        plt.ylabel('Peak Height (au)')
        plt.xlabel('m/z')
        plt.tight_layout()
        fig.savefig(os.path.join(folder_name,'%s_%s.pdf'%(data[cpd_id]['compound'],time_point)))
        fig.clear()
        plt.close('all')
plt.ion()

In [27]:
data[cpd_id]['compound'][0]

u'L-Carnosine'

In [38]:
df[df['original_index']==0]

,m/z,20171221_TS_DB_fullcyc_QE144_50454-738379_100_13C_Oxa_D3_1_POS_146.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_101_13C_Oxa_D3_2_POS_210.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_102_13C_Oxa_D3_3_POS_181.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_103_13C_Oxa_D6_1_POS_284.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_104_13C_Oxa_D6_2_POS_34.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_105_13C_Oxa_D6_3_POS_267.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_106_13C_Oxa_D14_1_POS_100.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_107_13C_Oxa_D14_2_POS_126.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_108_13C_Oxa_D14_3_POS_161.h5,...,20171221_TS_DB_fullcyc_QE144_50454-738379_96_13C_Lac_D3_1_POS_307.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_97_13C_Lac_D6_1_POS_123.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_98_13C_Lac_D6_2_POS_201.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_99_13C_Lac_D6_3_POS_B_380.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_9_12C_con_D3_3_POS_227.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_blank_60_POS_200.h5,20171221_TS_DB_fullcyc_QE144_50454-738379_blank_96_POS_318.h5,original_index,compound,time_point
0,118.0611,82769.3125,59444.644531,103061.703125,36695.691406,104442.390625,30431.396484,63663.945312,57787.003906,65324.449219,...,60036.097656,80161.578125,27808.035156,59547.625,31513.189453,3041.748779,2651.742188,0,glycocyamine,D14
0,118.0611,82769.3125,59444.644531,103061.703125,36695.691406,104442.390625,30431.396484,63663.945312,57787.003906,65324.449219,...,60036.097656,80161.578125,27808.035156,59547.625,31513.189453,3041.748779,2651.742188,0,glycocyamine,D30
0,118.0611,82769.3125,59444.644531,103061.703125,36695.691406,104442.390625,30431.396484,63663.945312,57787.003906,65324.449219,...,60036.097656,80161.578125,27808.035156,59547.625,31513.189453,3041.748779,2651.742188,0,glycocyamine,D48
0,118.0611,82769.3125,59444.644531,103061.703125,36695.691406,104442.390625,30431.396484,63663.945312,57787.003906,65324.449219,...,60036.097656,80161.578125,27808.035156,59547.625,31513.189453,3041.748779,2651.742188,0,glycocyamine,D6
0,118.0611,82769.3125,59444.644531,103061.703125,36695.691406,104442.390625,30431.396484,63663.945312,57787.003906,65324.449219,...,60036.097656,80161.578125,27808.035156,59547.625,31513.189453,3041.748779,2651.742188,0,glycocyamine,D3
0,118.0611,82769.3125,59444.644531,103061.703125,36695.691406,104442.390625,30431.396484,63663.945312,57787.003906,65324.449219,...,60036.097656,80161.578125,27808.035156,59547.625,31513.189453,3041.748779,2651.742188,0,glycocyamine,D1


In [51]:

filename = '/Users/bpb/Downloads/Archive (1)/negative_mode_isotope_data.pkl'
with open(filename,'rb') as fid:
    data = dill.load(fid)

for i,d in enumerate(data):
    data[i]['rt_peak'] = pd.DataFrame(data[i]['rt_peak'])
    data[i]['peak_height'] = pd.DataFrame(data[i]['peak_height'])
    
filename = '/Users/bpb/Downloads/Archive (1)/negative_mode_compound_data.pkl'
with open(filename,'rb') as fid:
    cpd_data = dill.load(fid)

all_dfs = []
for time_point in time_points:
    for cpd_id in range(len(data)):
        peak_height = data[cpd_id]['peak_height']
        peak_height.index.name='m/z'
        peak_height['compound'] = data[cpd_id]['compound'][0]
        peak_height['time_point'] = time_point
        cpd_dict = cpd_data[peak_height['original_index'].tolist()[0]]
        for k,v in cpd_dict.items():
            peak_height[k] = v
        all_dfs.append(peak_height.reset_index())
df = pd.concat(all_dfs)

df.to_csv('/Users/bpb/Downloads/neg_mode_isotope_table.csv',index=None)

In [47]:
df['chebi_id'].unique()

array([u'CHEBI:16344///CHEBI:57742', u'CHEBI:47977///CHEBI:58723',
       u'CHEBI:57757///CHEBI:28718///CHEBI:32828///CHEBI:58585',
       u'CHEBI:16737', u'CHEBI:17012///CHEBI:35418', u'CHEBI:27971',
       u'CHEBI:58360///CHEBI:18053///CHEBI:30526',
       u'CHEBI:49096///CHEBI:27389',
       u'CHEBI:356010///CHEBI:15887///CHEBI:86394',
       u'CHEBI:46911///CHEBI:15729///CHEBI:46914///CHEBI:44667',
       u'CHEBI:18107', u'CHEBI:17082///CHEBI:58672///CHEBI:37023',
       u'CHEBI:15698', u'CHEBI:18403',
       u'CHEBI:47031///CHEBI:16026///CHEBI:57609',
       u'CHEBI:18123///CHEBI:50521',
       u'CHEBI:18050///CHEBI:32665///CHEBI:32673///CHEBI:58000',
       u'CHEBI:32820///CHEBI:16857',
       u'CHEBI:70804///CHEBI:16974///CHEBI:57969', u'CHEBI:17256',
       u'CHEBI:66924///CHEBI:16919///CHEBI:57947',
       u'CHEBI:63475///CHEBI:58515///CHEBI:27592',
       u'CHEBI:32775///CHEBI:32760///CHEBI:32761///CHEBI:53696///CHEBI:17895///CHEBI:58570',
       u'CHEBI:17562', u'CHEBI:16349